In [1]:
import numpy as np
import pandas as pd
import os
import glob
import scipy 

In [2]:
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

In [3]:
# cSCC_counts = pd.read_csv(path+'scRNAseq/'+'GSE144236_cSCC_counts.txt', sep = '\t', chunksize=1000)

# # merge all chunks into one df
# scRNAseq = pd.concat([chunk for chunk in cSCC_counts])

# # split df by patient
# cSCC_scRNA = []
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     cSCC_scRNA.append(j)
    
# # save df by patient idex to separate CSV files
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     j.to_csv('Data/squamous_carcinoma/scRNAseq_{}.csv'.format(i), header=True, index_label=False)

In [4]:
# squamous carcinoma dataset
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

all_counts = pd.read_csv(path+'ST/'+'GSE144239_ST_all_counts.txt', sep = '\t')
st_barcodes = pd.read_csv(path+'ST/'+'GSE144239_ST_barcode_id.txt', sep = '\t', header=None)
st_prep_count = pd.read_csv(path+'ST/'+'GSE144239_ST_Visium_counts.txt', sep = '\t')

metadata = pd.read_csv(path+'scRNAseq/'+'GSE144236_patient_metadata_new.txt', sep = '\t')

In [5]:
# read in all patient scRNAseq files
scRNA = [pd.read_csv(j, index_col=0) for j in glob.glob(os.path.join(path, '*.csv'))]

In [6]:
all_counts

,P2_7x45_1,P2_8x30_1,P2_8x32_1,P2_8x34_1,P2_8x36_1,P2_8x38_1,P2_8x44_1,P2_8x46_1,P2_9x29_1,P2_9x31_1,...,P5_41x27_3,P5_41x31_3,P5_41x33_3,P5_41x35_3,P5_41x37_3,P5_41x39_3,P5_41x41_3,P5_42x36_3,P5_42x38_3,P5_42x40_3
Patient,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Section replicate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
pixel_x,3282.6,3473.4,3471.9,3474.1,3469.5,3467.4,3471.4,3462.3,3650.9,3649.9,...,9448.4,9446.6,9437.3,9451.4,9451.1,9445.3,9451.1,9634.2,9628.1,9623.4
pixel_y,9536.7,6539.5,6937.9,7338.8,7736.4,8137.2,9337.8,9738.9,6336.8,6737.0,...,6984.0,7809.0,8205.3,8613.4,9011.1,9412.5,9810.7,8815.8,9224.7,9620.7
FO538757.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAGEA2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MAGEA6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
LCE5A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NPIPA8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
np.unique(all_counts.loc['Patient'].values)

array([ 2.,  5.,  9., 10.])

In [17]:
np.unique(all_counts.loc['Section replicate'].values)

array([1., 2., 3.])

In [18]:
st_prep_count

,P4_AAACAAGTATCTCCCA-1_1,P4_AAACATTTCCCGGATT-1_1,P4_AAACCTAAGCAGCCGG-1_1,P4_AAACGAGACGGTTGAT-1_1,P4_AAACGGGCGTACGGGT-1_1,P4_AAAGGGATGTAGCAAG-1_1,P4_AAAGTGTGATTTATCT-1_1,P4_AAAGTTGACTCCCGTA-1_1,P4_AAATAGGGTGCTATTG-1_1,P4_AAATCCGATACACGCC-1_1,...,P6_TTGTGTATGCCACCAA-1_2,P6_TTGTGTTTCCCGAAAG-1_2,P6_TTGTTAGCAAATTCGA-1_2,P6_TTGTTCAGTGTGCTAC-1_2,P6_TTGTTGGCAATGACTG-1_2,P6_TTGTTGTGTGTCAAGA-1_2,P6_TTGTTTCACATCCAGG-1_2,P6_TTGTTTCCATACAACT-1_2,P6_TTGTTTGTATTACACG-1_2,P6_TTGTTTGTGTAAATTC-1_2
Patient,4,4,4,4,4,4,4,4,4,4,...,6,6,6,6,6,6,6,6,6,6
Section replicate,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
AL627309.1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL669831.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LINC00115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AP001059.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AP001469.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC136616.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC004556.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
st_barcodes

,0,1,2
0,GTCCGATATGATTGCCGC,2,6
1,GCGACGCGGCATAGTTTA,2,8
2,ACGTGCGCAAATATGACG,2,10
3,ATAAATGTTCACACTACT,2,12
4,TTGCTCGAATAAATAATT,2,14
...,...,...,...
1929,ATTAGTTATGCTGTTCAC,65,53
1930,GGTGAGACCTGAATTCCA,65,55
1931,CTATGTACTGTTCAAACC,65,57
1932,GGCTAGAACGAATAAACC,65,59


In [ ]:
print(np.unique(st_prep_count.iloc[0].values)) # find unique patients 
print(np.unique(st_prep_count.iloc[1].values)) # find unique replicates 

In [10]:
metadata

,nCount_RNA,nFeature_RNA,patient,tum.norm,level1_celltype,level2_celltype,level3_celltype
P1_Tumor_AAACCTGAGTCAAGCG,40856,5206,P1,Tumor,Epithelial,Tumor_KC_Diff,Tumor_KC_Diff
P1_Tumor_AAACCTGCAAATTGCC,15511,3468,P1,Tumor,Epithelial,Keratinocyte,Keratinocyte
P1_Tumor_AAACCTGGTAGGAGTC,24652,4154,P1,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P1_Tumor_AAACGGGAGATGTAAC,27554,4334,P1,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P1_Tumor_AAACGGGAGCTGCAAG,24980,4822,P1,Tumor,Epithelial,Tumor_KC_Cyc,Tumor_KC_Cyc
...,...,...,...,...,...,...,...
P10_Normal_TTTGTCAAGAGTCTGG,5585,1659,P10,Normal,Epithelial,Normal_KC_Diff,Normal_KC_Diff
P10_Normal_TTTGTCAAGTCGTTTG,14207,3541,P10,Normal,LC,LC,LC
P10_Normal_TTTGTCACAGTGGGAT,10068,2540,P10,Normal,LC,LC,LC
P10_Normal_TTTGTCATCCAGGGCT,5223,1849,P10,Normal,LC,LC,LC


In [ ]:
# count number of unique cells
barcds = [x.split('_')[2] for x in metadata.index] # split rownames on '_' and keep barcode only
np.unique(barcds).shape 

In [12]:
len(scRNA) # number of patients = 10 (as expected)

10